In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, accuracy_score, roc_auc_score
from glob import glob

RESULT_PATH = "results_m"

In [14]:
def ext_MPB(df_path):
    df_res = pd.read_csv(df_path)
    df_res = df_res[df_res.path.str.contains('MPB')]
    df_res[['true', 'fid', 'fcnt', 'dire', 'file']] = df_res.path.str.split('/', expand=True)[[3, 1, 2, 4, 5]]
    df_res.true = df_res.true.str.replace('unblink', '0').str.replace('blink', '1').str.replace('close', '1').str.replace('open', '0').astype(np.int32)
    df_res.pred = df_res.pred.astype(np.float64)
    df_res.file =df_res.file.str.replace('left', '').str.replace('right', '')

    return df_res

def ext_HUST(df_path):
    df_res = pd.read_csv(df_path)
    df_res = df_res[df_res.path.str.contains('HUST')]

    df_res[['true', 'fid', 'fcnt', 'dire']] = df_res.path.str.split('/', expand=True)[[2, 3, 4, 5]]
    df_res.true = df_res.true.str.replace('unblink', '0').str.replace('blink', '1').astype(np.int32)
    df_res.pred = df_res.pred.astype(np.float64)
    df_res.dire = df_res.dire.str.replace('zuo', 'left').replace('you', 'right')

    return df_res

def ext_mEBAL(df_path):
    df_res = pd.read_csv(df_path)
    df_res = df_res[df_res.path.str.contains('mEbal')]
    df_res[df_res.path.str.contains('/close/') | (df_res.path.str.contains('/open/') & df_res.path.str.contains('_1.jpg'))]
    df_res[['true', 'fid', 'dire', 'file']] = df_res.path.str.split('/', expand=True)[[2, 1, 3, 4]]
    df_res.true = df_res.true.str.replace('unblink', '0').str.replace('blink', '1').str.replace('close', '1').str.replace('open', '0').astype(np.int32)
    df_res.pred = df_res.pred.astype(np.float64)
    df_res.file =df_res.file.str.replace('left', '').str.replace('right', '')
    df_res.dire = df_res.dire.str.replace(' eye', '')
    df_res['fcnt'] = df_res.file.str.replace('jpg', '')
    return df_res

def evaluate(df):
    
    df_left = df[df['dire']=='left']
    df_right = df[df['dire']=='right']
    
    th = 0.5

    prec = [ precision_score(df.true, df.pred>th), precision_score(df_left.true, df_left.pred>th), precision_score(df_right.true, df_right.pred>th) ]
    recall = [ recall_score(df.true, df.pred>th), recall_score(df_left.true, df_left.pred>th), recall_score(df_right.true, df_right.pred>th) ]
    f1 = [ f1_score(df.true, df.pred>th), f1_score(df_left.true, df_left.pred>th), f1_score(df_right.true, df_right.pred>th) ]
    acc = [ accuracy_score(df.true, df.pred>th), accuracy_score(df_left.true, df_left.pred>th), accuracy_score(df_right.true, df_right.pred>th) ]
    aucs = [ roc_auc_score(df.true, df.pred), roc_auc_score(df_left.true, df_left.pred), roc_auc_score(df_right.true, df_right.pred) ]

    df_r = pd.DataFrame( np.array([prec, recall, f1, acc, aucs]).T, columns=['Precision', 'Recall', 'F1', 'Accuracy', 'AUC'])
    df_r['direction'] = [ 'Both', 'Left', 'Right' ]
    return df_r

def do(path):
    
    total_df = []
    
    df_hust = evaluate(ext_HUST(path))
    df_hust["model"] = [path.split('/')[1].split(".csv")[0][:-2]] * 3
    df_hust["iteration"] = [path.split('_')[-1].split('.csv')[0]] * 3
    df_hust["dataset"] = ['HUST'] * 3
    total_df+=[df_hust]

    df_me = evaluate(ext_mEBAL(path))
    df_me["model"] = [path.split('/')[1].split(".csv")[0][:-2]] * 3
    df_me["iteration"] = [path.split('_')[-1].split('.csv')[0]] * 3
    df_me["dataset"] = ['mEBAL'] * 3
    total_df+=[df_me]

    df_mpb_all = ext_MPB(path)
    df_mpb = evaluate(df_mpb_all)
    df_mpb["model"] = [path.split('/')[1].split(".csv")[0][:-2]] * 3
    df_mpb["iteration"] = [path.split('_')[-1].split('.csv')[0]] * 3
    df_mpb["dataset"] = ['MPB'] * 3
    total_df+=[df_mpb]
    
    for i in range(1, 10):
        df_mpb = evaluate(df_mpb_all[df_mpb_all.fcnt==str(i)])
        df_mpb["model"] = [path.split('/')[1].split(".csv")[0][:-2]] * 3
        df_mpb["iteration"] = [path.split('_')[-1].split('.csv')[0]] * 3
        df_mpb["dataset"] = ['MPB_{}'.format(i)] * 3
        total_df+=[df_mpb]    

    return pd.concat(total_df, axis=0, ignore_index=True)

In [ ]:
paths = glob("{}/*".format(RESULT_PATH))

res = []
for p in paths:
    res+= [do(p)]
    
res_final = pd.concat(res, axis=0, ignore_index=True)
res_final.groupby(['model', 'dataset', 'direction', ]).mean().merge(res_final.groupby(['model', 'dataset', 'direction', ]).std(), on = ['model', 'dataset', 'direction'], suffixes=['_mean', '_std']).to_csv('results.csv')